In [1]:
!pip install transformers==4.31.0 torch==2.0.1 scikit-learn==1.2.2 pandas==1.5.3 javalang==0.13.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 104.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 102.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [2]:
import os
import zipfile
import glob
import subprocess
import javalang
import torch
import numpy as np
import pandas as pd
from transformers import RobertaTokenizer, RobertaModel


In [3]:
from google.colab import files

# 1. Dataset Preparation
uploaded = files.upload()


Saving Promisedataset-main.zip to Promisedataset-main.zip


In [4]:
import zipfile
import os

with zipfile.ZipFile('Promisedataset-main.zip', 'r') as zip_ref:
    zip_ref.extractall('Promisedataset')

In [10]:
import os

# Check the current working directory
print("Current working directory:", os.getcwd())

# List the contents of the 'Promisedataset' directory
extracted_dir = 'Promisedataset'
for root, dirs, files in os.walk(extracted_dir):
    for name in dirs:
        print("Directory:", os.path.join(root, name))
    for name in files:
        print("File:", os.path.join(root, name))


Streaming output truncated to the last 5000 lines.
File: Promisedataset/Promisedataset-main/ant/ant/ant-1.7/apache-ant-1.7.0/src/main/org/apache/tools/ant/taskdefs/optional/ejb/JonasDeploymentTool.java
File: Promisedataset/Promisedataset-main/ant/ant/ant-1.7/apache-ant-1.7.0/src/main/org/apache/tools/ant/taskdefs/optional/ejb/WLRun.java
File: Promisedataset/Promisedataset-main/ant/ant/ant-1.7/apache-ant-1.7.0/src/main/org/apache/tools/ant/taskdefs/optional/ejb/BorlandDeploymentTool.java
File: Promisedataset/Promisedataset-main/ant/ant/ant-1.7/apache-ant-1.7.0/src/main/org/apache/tools/ant/taskdefs/optional/ejb/DescriptorHandler.java
File: Promisedataset/Promisedataset-main/ant/ant/ant-1.7/apache-ant-1.7.0/src/main/org/apache/tools/ant/taskdefs/optional/ejb/JbossDeploymentTool.java
File: Promisedataset/Promisedataset-main/ant/ant/ant-1.7/apache-ant-1.7.0/src/main/org/apache/tools/ant/taskdefs/optional/ejb/WeblogicTOPLinkDeploymentTool.java
File: Promisedataset/Promisedataset-main/ant/an

In [11]:
# Unrar if necessary
!apt-get install unrar

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [12]:
import os
import subprocess

def extract_rar_files(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".rar"):
                file_path = os.path.join(root, file)
                output_dir = os.path.join(root, os.path.splitext(file)[0])
                if not os.path.exists(output_dir):
                    os.makedirs(output_dir)
                subprocess.run(['unrar', 'x', file_path, output_dir])

extract_rar_files('Promisedataset/Promisedataset-main')

In [13]:
import os

# Dictionary to store project-wise Java files
project_files = {}

def list_java_files(directory):
    for root, dirs, files in os.walk(directory):
        # Filter to include only xalan directories
        if 'lucene-2.0.0-src' in root or 'lucene2.2.0-src' in root or 'lucene-2.4.0-src' in root:
            project_name = os.path.basename(root)
            if project_name not in project_files:
                project_files[project_name] = []
            for file in files:
                if file.endswith(".java"):
                    project_files[project_name].append(os.path.join(root, file))

# Update the directory path to your Promisedataset directory
list_java_files('Promisedataset/Promisedataset-main')

# Display the project-wise Java files for only the targeted versions
for project, files in project_files.items():
    if files:  # Only display projects with Java files
        print(f"Project: {project}")
        for file in files:
            print(f"  File: {file}")
        print(f"Total Java Files in {project}: {len(files)}\n")


Project: lucene
  File: Promisedataset/Promisedataset-main/lucene/lucene/lucene2.2.0-src/lucene2.2.0-src/lucene-2.2.0/src/test/org/apache/lucene/ThreadSafetyTest.java
  File: Promisedataset/Promisedataset-main/lucene/lucene/lucene2.2.0-src/lucene2.2.0-src/lucene-2.2.0/src/test/org/apache/lucene/SearchTestForDuplicates.java
  File: Promisedataset/Promisedataset-main/lucene/lucene/lucene2.2.0-src/lucene2.2.0-src/lucene-2.2.0/src/test/org/apache/lucene/SearchTest.java
  File: Promisedataset/Promisedataset-main/lucene/lucene/lucene2.2.0-src/lucene2.2.0-src/lucene-2.2.0/src/test/org/apache/lucene/AnalysisTest.java
  File: Promisedataset/Promisedataset-main/lucene/lucene/lucene2.2.0-src/lucene2.2.0-src/lucene-2.2.0/src/test/org/apache/lucene/IndexTest.java
  File: Promisedataset/Promisedataset-main/lucene/lucene/lucene2.2.0-src/lucene2.2.0-src/lucene-2.2.0/src/test/org/apache/lucene/StoreTest.java
  File: Promisedataset/Promisedataset-main/lucene/lucene/lucene2.2.0-src/lucene2.2.0-src/lucene

In [14]:
import os

# Define the paths for training, validation, and test sets for Lucene
train_paths = [
    "Promisedataset/Promisedataset-main/lucene/lucene/lucene-2.0.0-src/lucene-2.0.0-src/",
    "Promisedataset/Promisedataset-main/lucene/lucene/lucene2.2.0-src/lucene2.2.0-src/"
]
validation_path = "Promisedataset/Promisedataset-main/lucene/lucene/lucene2.2.0-src/lucene2.2.0-src/"
test_path = "Promisedataset/Promisedataset-main/lucene/lucene/lucene-2.4.0-src/lucene-2.4.0-src/"

# Verify that the paths exist
for path in train_paths:
    print(f"Train path {path} exists:", os.path.exists(path))
print("Validation path exists:", os.path.exists(validation_path))
print("Test path exists:", os.path.exists(test_path))


Train path Promisedataset/Promisedataset-main/lucene/lucene/lucene-2.0.0-src/lucene-2.0.0-src/ exists: True
Train path Promisedataset/Promisedataset-main/lucene/lucene/lucene2.2.0-src/lucene2.2.0-src/ exists: True
Validation path exists: True
Test path exists: True


In [15]:
import glob

# Function to get Java file paths
def get_java_files(directory):
    return glob.glob(os.path.join(directory, "**/*.java"), recursive=True)

# Load Java files into datasets
train_files = []
for path in train_paths:
    files = get_java_files(path)
    train_files.extend(files)
    print(f"Loaded {len(files)} training files from {path}")

validation_files = get_java_files(validation_path)
test_files = get_java_files(test_path)

print(f"Total training files: {len(train_files)}")
print(f"Loaded {len(validation_files)} validation files.")
print(f"Loaded {len(test_files)} test files.")


Loaded 727 training files from Promisedataset/Promisedataset-main/lucene/lucene/lucene-2.0.0-src/lucene-2.0.0-src/
Loaded 1249 training files from Promisedataset/Promisedataset-main/lucene/lucene/lucene2.2.0-src/lucene2.2.0-src/
Total training files: 1976
Loaded 1249 validation files.
Loaded 1288 test files.


In [16]:
from transformers import RobertaTokenizer

# Initialize the tokenizer for CodeBERT
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")

# Function to get Java file paths
import glob

def get_java_files(directory):
    return glob.glob(os.path.join(directory, "**/*.java"), recursive=True)

# Load Java files into datasets
train_files = [get_java_files(path) for path in train_paths]
validation_files = get_java_files(validation_path)
test_files = get_java_files(test_path)

# Flatten the train_files list
train_files = [item for sublist in train_files for item in sublist]

# Tokenize function for Java files
def tokenize_code(file_paths, tokenizer):
    inputs = []
    for file_path in file_paths:
        try:
            with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                code = f.read()
            input_ids = tokenizer.encode(code, truncation=True, padding='max_length', max_length=512)
            inputs.append(input_ids)
        except Exception as e:
            print(f"Error reading {file_path}: {e}")
    return inputs

# Tokenize train, validation, and test files
train_inputs = tokenize_code(train_files, tokenizer)
validation_inputs = tokenize_code(validation_files, tokenizer)
test_inputs = tokenize_code(test_files, tokenizer)

print("Tokenization complete!")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

Tokenization complete!


In [17]:
from torch.utils.data import Dataset

class CodeDataset(Dataset):
    def __init__(self, inputs):
        self.inputs = inputs
        self.labels = inputs  # Use inputs as labels for Masked Language Modeling

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return {
            'input_ids': self.inputs[idx],
            'labels': self.inputs[idx]
        }

train_dataset = CodeDataset(train_inputs)
validation_dataset = CodeDataset(validation_inputs)


In [18]:
from transformers import RobertaForMaskedLM, Trainer, TrainingArguments

model = RobertaForMaskedLM.from_pretrained("microsoft/codebert-base")

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    evaluation_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,No log,0.158446
2,No log,0.003609
3,2.640100,0.003002


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


TrainOutput(global_step=741, training_loss=1.783059175358771, metrics={'train_runtime': 1234.9536, 'train_samples_per_second': 4.8, 'train_steps_per_second': 0.6, 'total_flos': 1560637666418688.0, 'train_loss': 1.783059175358771, 'epoch': 3.0})

In [56]:
def get_embeddings(inputs):
    embeddings = []
    model.eval()  # Switch to evaluation mode
    with torch.no_grad():
        for input_ids in inputs:
            input_ids = torch.tensor(input_ids).unsqueeze(0).to(model.device)  # Add batch dimension
            attention_mask = torch.tensor([1] * len(input_ids[0])).unsqueeze(0).to(model.device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)

            # Extract logits instead of `last_hidden_state`
            logits = outputs.logits  # Use logits instead, as 'MaskedLMOutput' returns logits
            embedding = logits.mean(dim=1).squeeze().cpu().numpy()  # Mean pooling over sequence length
            embeddings.append(embedding)
    return np.array(embeddings)


In [57]:
# Generate embeddings for training, validation, and test sets
train_embeddings = get_embeddings(train_inputs)
validation_embeddings = get_embeddings(validation_inputs)
test_embeddings = get_embeddings(test_inputs)

print("Embeddings generated.")


Embeddings generated.


In [58]:
print(f"Train files: {len(train_files)}")
print(f"Validation files: {len(validation_files)}")
print(f"Test files: {len(test_files)}")


Train files: 1976
Validation files: 1249
Test files: 1288


In [74]:
print(f"Train embeddings shape: {train_embeddings.shape}")
print(f"Validation embeddings shape: {validation_embeddings.shape}")
print(f"Test embeddings shape: {test_embeddings.shape}")


Train embeddings shape: (1976, 50265)
Validation embeddings shape: (1249, 50265)
Test embeddings shape: (1288, 50265)


In [84]:
from google.colab import files
import pandas as pd

# Upload and load the CSV files for Lucene versions
uploaded = files.upload()

# Assuming the files are named "lucene_2_0.csv", "lucene_2_2.csv", and "lucene_2_4.csv"
lucene_2_0_df = pd.read_csv('lucene-2.0.csv')
lucene_2_2_df = pd.read_csv('lucene-2.2.csv')
lucene_2_4_df = pd.read_csv('lucene-2.4.csv')

# Use the second 'name' column and the 'bug' column explicitly by their column index
# Assuming the second 'name' column is in position 2
lucene_2_0_labels = lucene_2_0_df.iloc[:, [2, lucene_2_0_df.columns.get_loc('bug')]]
lucene_2_2_labels = lucene_2_2_df.iloc[:, [2, lucene_2_2_df.columns.get_loc('bug')]]
lucene_2_4_labels = lucene_2_4_df.iloc[:, [2, lucene_2_4_df.columns.get_loc('bug')]]

# Check the labels
print(lucene_2_0_labels.head())
print(lucene_2_2_labels.head())
print(lucene_2_4_labels.head())


Saving lucene-2.0.csv to lucene-2.0 (5).csv
Saving lucene-2.2.csv to lucene-2.2 (5).csv
Saving lucene-2.4.csv to lucene-2.4 (5).csv
                                          name.1  bug
0  org.apache.lucene.analysis.WhitespaceAnalyzer    0
1       org.apache.lucene.search.QueryTermVector    0
2       org.apache.lucene.analysis.PorterStemmer    0
3          org.apache.lucene.index.TermPositions    3
4         org.apache.lucene.analysis.TokenStream    1
                                             name.1  bug
0               org.apache.lucene.search.FuzzyQuery    1
1  org.apache.lucene.index.MultiLevelSkipListReader    1
2              org.apache.lucene.store.RAMDirectory    2
3          org.apache.lucene.search.spans.SpanQuery    0
4             org.apache.lucene.index.TermPositions    1
                                          name.1  bug
0   org.apache.lucene.index.DocInverterPerThread    1
1            org.apache.lucene.search.FuzzyQuery    3
2       org.apache.lucene.search.QueryTe

In [85]:
import os

def get_defect_labels(file_paths, labels_df):
    labels = []
    for file_path in file_paths:
        file_name = os.path.basename(file_path).replace(".java", "")
        label_row = labels_df[labels_df.iloc[:, 0].str.contains(file_name)]  # Match on the second 'name' column
        if not label_row.empty:
            labels.append(int(label_row['bug'].values[0]))
        else:
            labels.append(None)
    return labels

# Get labels for train, validation, and test datasets
train_labels_2_0 = get_defect_labels([f for f in train_files if f.startswith(train_paths[0])], lucene_2_0_labels)
train_labels_2_2 = get_defect_labels([f for f in train_files if f.startswith(train_paths[1])], lucene_2_2_labels)
train_labels = train_labels_2_0 + train_labels_2_2
validation_labels = get_defect_labels(validation_files, lucene_2_2_labels)  # Assuming validation for 2.2
test_labels = get_defect_labels(test_files, lucene_2_4_labels)  # Assuming test for 2.4

In [86]:
# Filter embeddings and labels as done earlier
def filter_data(embeddings, labels):
    filtered_embeddings = []
    filtered_labels = []
    for emb, label in zip(embeddings, labels):
        if label is not None:
            filtered_embeddings.append(emb)
            filtered_labels.append(label)
    return np.array(filtered_embeddings), filtered_labels

# Assuming embeddings already generated
train_embeddings_filtered, y_train = filter_data(train_embeddings, train_labels)
validation_embeddings_filtered, y_validation = filter_data(validation_embeddings, validation_labels)
test_embeddings_filtered, y_test = filter_data(test_embeddings, test_labels)

# Now save the filtered embeddings
np.save("lucene_train_embeddings_codebert.npy", train_embeddings_filtered)
np.save("lucene_validation_embeddings_codebert.npy", validation_embeddings_filtered)
np.save("lucene_test_embeddings_codebert.npy", test_embeddings_filtered)

print("Filtered embeddings and labels saved.")

Filtered embeddings and labels saved.


In [87]:
# Load CodeBERT embeddings for Lucene
codebert_vectors = np.load("lucene_test_embeddings_codebert.npy")
pca_codebert = PCA(n_components=256)
codebert_vectors_pca = pca_codebert.fit_transform(codebert_vectors)
print(f"CodeBERT embeddings shape after PCA: {codebert_vectors_pca.shape}")

CodeBERT embeddings shape after PCA: (688, 256)


### **Comparision**

In [88]:
# Load Code2Vec embeddings (adjust file name if needed)
code2vec_vectors = np.loadtxt('lucene_dataset.test.c2v.vectors')

# Apply PCA to Code2Vec vectors
from sklearn.decomposition import PCA
pca = PCA(n_components=256)
code2vec_vectors_pca = pca.fit_transform(code2vec_vectors)
print(f"Code2Vec embeddings shape after PCA: {code2vec_vectors_pca.shape}")

Code2Vec embeddings shape after PCA: (3000, 256)


In [90]:
# Ensure embeddings and labels are aligned for Lucene
valid_indices_codebert = [i for i, label in enumerate(lucene_2_4_labels['bug']) if pd.notna(label)]
valid_indices_code2vec = [i for i, label in enumerate(lucene_2_4_labels['bug']) if pd.notna(label)]

# Use valid indices to extract embeddings and labels for Lucene
X_codebert = codebert_vectors_pca[valid_indices_codebert]
y_codebert = [int(lucene_2_4_labels['bug'][i]) for i in valid_indices_codebert]

X_code2vec = code2vec_vectors_pca[valid_indices_code2vec]
y_code2vec = [int(lucene_2_4_labels['bug'][i]) for i in valid_indices_code2vec]

# Ensure the number of valid embeddings matches the number of labels for Lucene
print(f"Final CodeBERT embeddings: {len(X_codebert)}, labels: {len(y_codebert)}")
print(f"Final Code2Vec embeddings: {len(X_code2vec)}, labels: {len(y_code2vec)}")


Final CodeBERT embeddings: 340, labels: 340
Final Code2Vec embeddings: 340, labels: 340


In [91]:
# Split CodeBERT embeddings and labels
X_train_codebert, X_test_codebert, y_train_codebert, y_test_codebert = train_test_split(
    X_codebert, y_codebert, test_size=0.2, random_state=42)

# Split Code2Vec embeddings and labels
X_train_code2vec, X_test_code2vec, y_train_code2vec, y_test_code2vec = train_test_split(
    X_code2vec, y_code2vec, test_size=0.2, random_state=42)

print(f"CodeBERT - Train set size: {len(X_train_codebert)}, Test set size: {len(X_test_codebert)}")
print(f"Code2Vec - Train set size: {len(X_train_code2vec)}, Test set size: {len(X_test_code2vec)}")


CodeBERT - Train set size: 272, Test set size: 68
Code2Vec - Train set size: 272, Test set size: 68


In [92]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Initialize RandomForest for CodeBERT
clf_codebert = RandomForestClassifier(random_state=42)
clf_codebert.fit(X_train_codebert, y_train_codebert)

# Make predictions on the CodeBERT test set
y_pred_codebert = clf_codebert.predict(X_test_codebert)

# Evaluate CodeBERT's performance
accuracy_codebert = accuracy_score(y_test_codebert, y_pred_codebert)
precision_codebert = precision_score(y_test_codebert, y_pred_codebert, average='weighted', zero_division=0)
recall_codebert = recall_score(y_test_codebert, y_pred_codebert, average='weighted', zero_division=0)
f1_codebert = f1_score(y_test_codebert, y_pred_codebert, average='weighted', zero_division=0)

print(f"CodeBERT - Accuracy: {accuracy_codebert:.4f}")
print(f"CodeBERT - Precision: {precision_codebert:.4f}")
print(f"CodeBERT - Recall: {recall_codebert:.4f}")
print(f"CodeBERT - F1-Score: {f1_codebert:.4f}")


CodeBERT - Accuracy: 0.4118
CodeBERT - Precision: 0.1721
CodeBERT - Recall: 0.4118
CodeBERT - F1-Score: 0.2427


In [93]:
# Initialize RandomForest for Code2Vec
clf_code2vec = RandomForestClassifier(random_state=42)
clf_code2vec.fit(X_train_code2vec, y_train_code2vec)

# Make predictions on the Code2Vec test set
y_pred_code2vec = clf_code2vec.predict(X_test_code2vec)

# Evaluate Code2Vec's performance
accuracy_code2vec = accuracy_score(y_test_code2vec, y_pred_code2vec)
precision_code2vec = precision_score(y_test_code2vec, y_pred_code2vec, average='weighted', zero_division=0)
recall_code2vec = recall_score(y_test_code2vec, y_pred_code2vec, average='weighted', zero_division=0)
f1_code2vec = f1_score(y_test_code2vec, y_pred_code2vec, average='weighted', zero_division=0)

print(f"Code2Vec - Accuracy: {accuracy_code2vec:.4f}")
print(f"Code2Vec - Precision: {precision_code2vec:.4f}")
print(f"Code2Vec - Recall: {recall_code2vec:.4f}")
print(f"Code2Vec - F1-Score: {f1_code2vec:.4f}")


Code2Vec - Accuracy: 0.3235
Code2Vec - Precision: 0.1810
Code2Vec - Recall: 0.3235
Code2Vec - F1-Score: 0.2309
